# 1)

In [4]:
import mysql.connector
def main():
    host = "bank.cw2w4gcvc1md.us-east-1.rds.amazonaws.com"
    user = "Ankur"
    password = open("credentials.txt").read()
    conn = mysql.connector.connect(host=host, 
                                    user=user, 
                                    password = password)
    if conn.is_connected() == True:
        print("Connection Successful")
    conn.close()  
main()

Connection Successful


In [48]:
import mysql.connector
import csv

def main():
    host = "bank.cw2w4gcvc1md.us-east-1.rds.amazonaws.com"
    user = "Ankur"
    password = open("credentials.txt").read()
    count2=0;
    tables = ["branch.csv","account.csv","customer.csv","loan.csv", "depositor.csv", "borrower.csv"]
    sql1 = """
    INSERT INTO account(account_number, branch_name, balance) VALUES(%s, %s, %s);"""
    sql2 = """
    INSERT INTO loan(loan_number, branch_name, amount) VALUES(%s, %s, %s);"""
    sql3 = """
    INSERT INTO branch(branch_name, branch_city, assets) VALUES(%s, %s, %s);"""
    sql4 = """
    INSERT INTO customer(customer_name, customer_street, customer_city) VALUES(%s, %s, %s);"""
    sql5 = """
    INSERT INTO depositor(customer_name, account_number) VALUES(%s, %s);"""
    sql6 = """
    INSERT INTO borrower(customer_name, loan_number) VALUES(%s, %s);"""
    
    conn = mysql.connector.connect(host=host, 
                                    user=user, 
                                    password = password,
                                    database='cse4020_db')
    if conn.is_connected() == True:
        print("Connection Successful")
    cursor=conn.cursor()
    try:    
        for table in tables:
            count = 0
            with open(table,'r') as csvfile:

                reader = csv.reader(csvfile)
                next(reader)
                for record in csvfile:
                    if table == "branch.csv":

                        branch_name, branch_city, assets = record.split(',')
                        assets = assets.rstrip()
                        #print(branch_name+branch_city+assets)
                        row = (branch_name, branch_city, assets)   
                        cursor.execute(sql3, row)
                        count += 1


                    if table == "account.csv":

                        cursor.execute("ALTER TABLE account MODIFY COLUMN account_number VARCHAR(40);")
                        account_number, branch_name, balance = record.split(',')
                        balance = balance.rstrip()
                        row = (account_number, branch_name, balance)
                        #print(account_number+' '+branch_name+' '+balance)
                        cursor.execute(sql1, row)
                        count += 1



                    if table == "loan.csv":

                        cursor.execute("ALTER TABLE loan MODIFY COLUMN loan_number VARCHAR(40);")

                        loan_number, branch_name, amount = record.split(',')
                        amount = amount.rstrip()
                        row = (loan_number, branch_name, amount)
                        #print(loan_number+' '+branch_name+' '+amount)
                        cursor.execute(sql2, row)
                        count += 1


                    if table == "customer.csv":

                        customer_name, customer_street, customer_city = record.split(',')
                        customer_city = customer_city.rstrip()
                        row = (customer_name, customer_street, customer_city)
                        #print(customer_name+' '+customer_street+' '+customer_city)
                        cursor.execute(sql4, row)
                        count += 1


                    if table == "depositor.csv":
                        cursor.execute("ALTER TABLE depositor MODIFY COLUMN account_number VARCHAR(40);")
                        customer_name, account_number = record.split(',')
                        account_number = account_number.rstrip()
                        row = (customer_name, account_number)
                        #print(customer_name+' '+account_number)
                        cursor.execute(sql5, row)
                        count += 1


                    if table == "borrower.csv":
                        cursor.execute("ALTER TABLE borrower MODIFY COLUMN loan_number VARCHAR(40);")
                        customer_name, loan_number = record.split(',')
                        loan_number = loan_number.rstrip()
                        row = (customer_name, loan_number)
                        #print(customer_name+' '+loan_number)
                        cursor.execute(sql6, row)
                        count += 1

                count2+=1;
                print(f"{count} record(s) inserted into the {table} table")
        
                

        conn.commit()
        print("Data added successfully")
        
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
    conn.close()
        


main()


    

Connection Successful
9 record(s) inserted into the branch.csv table
9 record(s) inserted into the account.csv table
15 record(s) inserted into the customer.csv table
9 record(s) inserted into the loan.csv table
10 record(s) inserted into the depositor.csv table
10 record(s) inserted into the borrower.csv table
Data added successfully


# 2)

```mysql
     SELECT
             customer.customer_name,
             customer.customer_city,
             depositor.account_number,
             branch.branch_name,
             concat('$',format(branch.assets,2)) as assets
    FROM customer
        JOIN depositor on depositor.customer_name=customer.customer_name
        JOIN account on account.account_number=depositor.account_number
        JOIN branch on branch.branch_name=account.branch_name
    WHERE branch.assets >= 3000000
    ORDER BY branch.assets DESC

```

In [59]:
def main():
    host = "bank.cw2w4gcvc1md.us-east-1.rds.amazonaws.com"
    user = "Ankur"
    password = open("credentials.txt").read()
    
    conn = mysql.connector.connect(host=host, 
                                    user=user, 
                                    password = password,
                                    database='cse4020_db')
    cursor=conn.cursor()
    
    sql_query="""
    Select customer.customer_name,customer.customer_city,depositor.account_number,branch.branch_name,concat('$',format(branch.assets,2)) as assests
    From customer
    join depositor on depositor.customer_name=customer.customer_name
    join account on account.account_number=depositor.account_number
    join branch on branch.branch_name=account.branch_name
    where branch.assets >= 3000000
    order by branch.assets desc"""
    cursor.execute(sql_query)
    store=cursor.fetchall()
    print(f'{"customer_name":20}{"customer_city":25}{"account_number":20}{"branch_name":20}{"assets":20}')
    for rows in store:
        
        print(f'{rows[0]:20}{rows[1]:25}{rows[2]:20}{rows[3]:20}{rows[4]:20}')
    conn.close()    
main()

customer_name       customer_city            account_number      branch_name         assets              
Turner              Stamford                 A-305               Round Hill          $8,000,000.00       
Jones               Harrison                 A-217               Brighton            $7,000,000.00       
Smith               Rye                      A-444               North Town          $3,700,000.00       
